#### **1. 设置环境**

导入 Python 标准库、第三方库和本项目自定义库

In [14]:
# 标准库
import os
import sys
import json

# 第三方库
# pip install pandas tqdm
import pandas as pd
from tqdm import tqdm

# 将上级目录加入系统路径
# 以便导入项目自定义库
sys.path.append(os.path.abspath('..'))

# 自定义库
# 大模型机器翻译模块
from src.utils import load_corpus
from src.annotator.mt_generation import MTGenerator

#### **2. 读取语料**

加载 TSV 格式的平行语料库

In [15]:
# 指定语料库目录 data/raw
# 目录中包含两个 TSV 文件：
# test_data_001.tsv 和 test_data_002.tsv

# TSV 文件格式：
# 第一列：汉语原文，选自《鹿鼎记》第一回和第十回
# 第二列：英语译文，选自《The Deer and The Cauldron》（闵福德译）

data_dir = '../data/raw'
print(f"从 {data_dir} 目录读取数据 ...")

data = load_corpus(data_dir)
print(f"成功读取 {len(data)} 条平行句对")

# 筛选数据：
# 设置 LIMIT=10，仅翻译前 10 句中文
# 设置 LIMIT=None，翻译全部中文
# 若数据较多，可通过 LIMIT 变量控制大模型 API 成本
LIMIT = 10
#LIMIT = None
if LIMIT:
    data = data.head(LIMIT)
print(f"准备为前 {len(data)} 句中文生成英文翻译")

# 预览数据：
# 导入后的数据以 DataFrame 格式存储
# 第一列：原文；第二列：英文；第三列：文件名
# 原文为 None，表示增译
# 译文为 None，表示减译
print("数据前 5 行如下：")
data.head()

从 ../data/raw 目录读取数据 ...
成功读取 305 条平行句对
准备为前 10 句中文生成英文翻译
数据前 5 行如下：


,source_text,target_text,file_id
0,北风如刀，满地冰霜。,None,test_data_001.tsv
1,江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。,Along a coastal road somewhere south of the Ya...,test_data_001.tsv
2,前面三辆囚车中分别监禁的是三个男子，都作书生打扮，一个是白发老者，两个是中年人。,In each of the first three carts a single male...,test_data_001.tsv
3,后面四辆囚车中坐的是女子，最后一辆囚车中是个少妇，怀中抱着个女婴。,"The four rear carts were occupied by women, th...",test_data_001.tsv
4,女婴啼哭不休。 她母亲温言相呵，女婴只是大哭。,The little girl was crying in a continuous wai...,test_data_001.tsv


#### **3. 加载模型**

加载大模型机器翻译模型

In [5]:
# 指定用于机器翻译的大模型
# 候选模型：
# qwen-flash, qwen-plus, qwen3-max, glm-4.7, deepseek-v3.2

# model_names 列表可选取多个模型
# 生成包含多版本译文的平行语料库

model_names = ['deepseek-v3.2', 'qwen3-max']

# 指定原文和译文语种
# 用于构建翻译任务提示词
# 提示词模版详见：
# src/prompt/mt_generation_prompt.py

src_lang = 'Chinese'
tgt_lang = 'English'

print("加载翻译模型 ...")
print(f"翻译方向：{src_lang} -> {tgt_lang}")

# 加载模型前，请登录阿里云百炼平台：https://bailian.console.aliyun.com/
# 申请调用大模型服务的 API 账号
# 并在 llm_corpus_annotation/.env 文件中设置 LLM_API_KEY=sk-********

models = {}
for model_name in model_names:
    models[model_name] = MTGenerator(model=model_name)
loaded_models = list(models.keys())

print(f"模型加载完成！使用以下 {len(loaded_models)} 个模型生成译文：")
print(f"{('\n').join(loaded_models)}")

加载翻译模型 ...
翻译方向：Chinese -> English
模型加载完成！使用以下 2 个模型生成译文：
deepseek-v3.2
qwen3-max


#### **4. 批量翻译**

使用 DeepSeek-V3.2 和 Qwen3-Max 大模型，自动生成机器译文

构建包含人类和机器译文的多版本平行语料库

In [6]:
# === 核心函数：translate_data ===
# 调用大模型 API 批量翻译原文，结果保存于 results 列表

# 参数 data：DataFrame 格式的数据（详见步骤 2）
# 参数 models：大模型 API 接口（详见步骤 3）

def translate_data(data, models):
    results = []

    # 逐行遍历所有数据
    for index, row in tqdm(data.iterrows(), total=len(data), desc="Translating"):

        # 提取汉语原文和人类译文
        src_text = row["source_text"]
        human_trans = row["target_text"]

        # 初始化 record 字典
        # 用于保存大模型翻译结果
        record = {
            "id": f"{index:06d}",
            "source": src_text,
            "targets": {
                "human": human_trans,
            }
        }

        for model in models.keys():
            record["targets"][model] = None

        # 若原文为空（增译），直接返回初始 
        if not src_text:
            results.append(record)
        # 否则，调用大模型 API 翻译原文
        else:
            try:
                # 按所选模型（deepseek-v3.2 和 qwen3-max）
                # 依次生成不同版本的机器译文
                for model in models.keys():
                    model_trans = models[model].translate(src_text)
                    record["targets"][model] = model_trans['target_text']
                # 将翻译结果保存于 results 列表
                results.append(record)
            except Exception as e:
                print(f"Error at index {index}: {e}")
                continue
                
    return results

In [7]:
# 注意：
# 为节省 API 调用成本，本项目将大模型生成内容保存于本地缓存 data/llm_cache
# 完成首次调用后，再次调用只需从本地数据库读取生成结果
# 因此，程序运行时间显示为 0.0 秒

# 首次调用 API 翻译 305 句原文
# 生成 2 个大模型译本（deepseek-v3.2 + qwen3-max）
# 大约需要 20 分钟

# 开始翻译
results = translate_data(data, models)

Translating: 100%|████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 724.69it/s]


#### **5. 保存结果**

将大模型译文保存于 JSON 文件

In [8]:
# save_results 函数：
# 将 results 列表转换为 JSON 格式
# 保存于 out_file 文件
def save_results(results, out_file):
     with open(out_file, "wt", encoding="utf-8") as fout:
        for record in results:
            fout.write(json.dumps(record, ensure_ascii=False) + "\n")

In [9]:
# 指定输出文件
if LIMIT:
    out_file = f"../data/output/0_mt_generation_limit{LIMIT}.jsonl"
else:
    out_file = f"../data/output/0_mt_generation.jsonl"
    
# 保存大模型译文
save_results(results, out_file)
print(f"结果已保存至 {out_file}\n")

结果已保存至 ../data/output/0_mt_generation_limit10.jsonl



#### **6. 查看结果**

预览大模型译文，查看人类-机器译文异同

In [11]:
# 预览前 3 条数据
# 输出人类译文、Deepseek译文和 Qwen3-Max 译文
for res in results[:3]:
    print(f"\n[ID]: {res['id']}")
    print(f"【原文】: {res['source']}")
    print("-" * 60)
    print(f"【Human】   : {res['targets']['human']}")
    print(f"【DeepSeek】: {res['targets']['deepseek-v3.2']}")
    print(f"【Qwen-Max】: {res['targets']['qwen3-max']}")
    print("=" * 60)


[ID]: 000000
【原文】: 北风如刀，满地冰霜。
------------------------------------------------------------
【Human】   : None
【DeepSeek】: The north wind cut like a knife, and frost covered the ground.
【Qwen-Max】: The north wind cuts like a knife; ice and frost cover the ground.

[ID]: 000001
【原文】: 江南近海滨的一条大路上，一队清兵手执刀枪，押着七辆囚车，冲风冒寒，向北而行。
------------------------------------------------------------
【Human】   : Along a coastal road somewhere south of the Yangtze River, a detachment of soldiers, each of them armed with a halberd, was escorting a line of seven prison carts, trudging northwards in the teeth of a bitter wind.
【DeepSeek】: Along a broad road near the coast south of the Yangtze River, a troop of Qing soldiers, armed with swords and spears, escorted seven prison carts northward, braving the biting wind and cold.
【Qwen-Max】: On a major road near the coast in Jiangnan, a squad of Qing soldiers, armed with swords and spears, escorted seven prison carts northward through biting wind and bitter cold.

